In [1]:
import pandas as pd
import pylab as pl
import numpy as np
import umap
from umap.umap_ import nearest_neighbors

In [2]:
tng_photo  = pd.read_csv('./data/tng_features.csv').drop(['Unnamed: 0'], axis=1)
sdss_photo = pd.read_csv('./data/stellar_sdss_phot.csv').drop(['Unnamed: 0'], axis=1)
tweb_df = pd.read_csv('./data/tweb_cw_grid128_rcellsize.csv').drop(['Unnamed: 0'], axis=1)

In [3]:
df = pd.merge(tng_photo, tweb_df, on='ID')
df

,ID,pos_x,pos_y,pos_z,vel_x,vel_y,vel_z,spin_x,spin_y,spin_z,...,tng_MK,tng_Mg,tng_Mr,tng_Mi,tng_Mz,lambda1,lambda2,lambda3,delta,Environment
0,0.0,20130.394531,47349.105469,48801.187500,108.748199,492.770233,-79.742752,-2001.918091,-44744.988281,-56517.378906,...,-2.648954e+01,-2.369290e+01,-2.422302e+01,-2.449123e+01,-2.467778e+01,5.767253,8.096545,11.461313,84.826370,0.0
1,1.0,20396.642578,48362.144531,48838.035156,-357.675751,-584.611694,132.533401,-202.446609,-5010.732910,6003.585449,...,-2.736132e+01,-2.407614e+01,-2.481237e+01,-2.516002e+01,-2.540629e+01,4.591577,6.306723,10.207852,52.117297,0.0
2,2.0,20295.615234,46623.386719,49167.097656,-620.661072,817.448364,101.115845,-1089.893188,-153.422882,-5759.730957,...,-2.687820e+01,-2.391769e+01,-2.446360e+01,-2.474424e+01,-2.495579e+01,2.507324,6.697186,9.044417,37.486743,0.0
3,3.0,18736.601562,48797.214844,49059.656250,385.925079,-0.796322,254.070755,-1529.094116,4462.931152,-2068.250244,...,-2.523255e+01,-2.227301e+01,-2.281964e+01,-2.310574e+01,-2.331848e+01,-1.432878,4.136361,5.670569,18.289966,1.0
4,4.0,19634.787109,46360.535156,48662.789062,452.051331,915.428833,-752.856201,-1473.783325,339.460846,-1401.718750,...,-2.548006e+01,-2.216172e+01,-2.290466e+01,-2.325472e+01,-2.350575e+01,2.242271,5.716625,8.317782,25.372272,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4527122,4527122.0,71944.601562,41649.585938,14307.522461,189.576904,-253.501907,-10.517840,-3.235120,-4.793508,2.107407,...,1.000000e+37,1.000000e+37,1.000000e+37,1.000000e+37,1.000000e+37,-0.106685,-0.082913,0.003001,-0.906812,2.0
4527123,4527123.0,72408.265625,34944.980469,70271.257812,228.023682,-138.439804,176.395828,1.310551,-6.123199,3.143746,...,1.000000e+37,1.000000e+37,1.000000e+37,1.000000e+37,1.000000e+37,-0.135545,-0.027500,0.212230,0.025071,2.0
4527124,4527124.0,73568.648438,25572.833984,13796.928711,432.669159,197.056992,606.003723,0.343861,0.976535,10.269505,...,1.000000e+37,1.000000e+37,1.000000e+37,1.000000e+37,1.000000e+37,-0.425395,0.193305,1.047978,-0.440871,1.0
4527125,4527125.0,71121.132812,27276.480469,3025.910400,192.559647,235.901352,230.489120,-6.148301,-7.261300,-2.401796,...,1.000000e+37,1.000000e+37,1.000000e+37,1.000000e+37,1.000000e+37,-0.163914,0.055180,0.270512,-0.440871,1.0


In [4]:
df.keys()

Index(['ID', 'pos_x', 'pos_y', 'pos_z', 'vel_x', 'vel_y', 'vel_z', 'spin_x',
       'spin_y', 'spin_z', 'tng_MU', 'tng_MB', 'tng_MV', 'tng_MK', 'tng_Mg',
       'tng_Mr', 'tng_Mi', 'tng_Mz', 'lambda1', 'lambda2', 'lambda3', 'delta',
       'Environment'],
      dtype='object')

In [5]:
predictors = ['tng_MU', 'tng_MB', 'tng_MV', 'tng_MK', 'tng_Mg', 'tng_Mr', 'tng_Mi', 'tng_Mz']
data =df[predictors]

In [8]:
n_neighbors = [5, 50, 100, 250]
min_dists = [0, 0.2, 0.5, 0.9]
normal_embeddings = np.zeros((len(min_dists), len(n_neighbors), len(data), 2))
precomputed_knn_embeddings = np.zeros((len(min_dists), len(n_neighbors), len(data), 2))

In [ ]:
# Knn precomputed
knn = nearest_neighbors(data, n_neighbors=250, metric="euclidean", metric_kwds=None, angular=False, random_state=None)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [ ]:
# Features map using knn precpmputed
for i, k in enumerate(n_neighbors):
    for j, dist in enumerate(min_dists):
        precomputed_knn_embeddings[i, j] = umap.UMAP(n_neighbors=k,
                                                      min_dist=dist,
                                                      precomputed_knn=knn,
                                                      ).fit_transform(data)

In [ ]:
fig, axs = pl.subplots(4, 4, figsize=(20, 20))

for i, ax_row in enumerate(axs):
    for j, ax in enumerate(ax_row):
        ax.scatter(precomputed_knn_embeddings[i, j, :, 0],
                   precomputed_knn_embeddings[i, j, :, 1],
                   c=labels / 9,
                   cmap='tab10',
                   alpha=0.1,
                   s=1,
                   )
        ax.set_xticks([])
        ax.set_yticks([])
        if i == 0:
            ax.set_title("min_dist = {}".format(min_dists[j]), size=15)
        if j == 0:
            ax.set_ylabel("n_neighbors = {}".format(n_neighbors[i]), size=15)
fig.suptitle("UMAP embedding of MNIST digits with grid of parameters", y=0.92, size=20)
pl.subplots_adjust(wspace=0.05, hspace=0.05)